# Egobox

## Installation

You have to get the right wheel archive regarding your Python/Platform installation.

Then you can install with the pip command as follows: (here for Python 3.8 on Windows)

pip install egobox-0.1.0-cp38-none-win_amd64.whl

## Import

In [1]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Test functions 

In [2]:
xlimits_xsinx = np.array([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [3]:
xlimits_g24 = np.array([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


## Egobox Rust optimizer (a.k.a Egor)

In [4]:
egor = egx.Optimizer(g24, xlimits_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     infill_strategy=egx.InfillStrategy.WB2,
                     expected=egx.ExpectedOptimum(val=-5.50, tol=1e-2),
                     outdir="./out",
                     hot_start=True
                    )  # see help(SegoOptimizer) for options

# Restrict regression and correlation models used
#egor = egx.Optimizer(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [9]:
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:ego.egor:Compute initial LHS with 10 points
INFO:ego.egor:Save initial doe in "./out\\egor_initial_doe.npy"
INFO:ego.egor:Start for 20 iterations
INFO:moe.algorithm:Best expert Linear_Matern52 accuracy=0.0000000000000001554312234475219
INFO:moe.algorithm:Best expert -> Linear_Matern52
INFO:moe.algorithm:Best expert Linear_SquaredExponential accuracy=1.0518684740641633
INFO:moe.algorithm:Best expert -> Linear_SquaredExponential
INFO:moe.algorithm:Best expert Linear_Matern52 accuracy=1.295503487936383
INFO:moe.algorithm:Best expert -> Linear_Matern52
INFO:ego.egor:Add 1 point:
INFO:ego.egor:  [[2.331734247313595, 3.3236700516833717]], shape=[1, 2], strides=[2, 1], layout=CFcf (0xf), const ndim=2
INFO:ego.egor:Save doe in "./out\\egor_doe.npy"
INFO:ego.egor:Iteration 1/20: Best fun(x)=[-3.932803705812783, -0.08929177637534513, -1.67437722939286] at x=[1.8313879796937838, 2.1014157261189994]
INFO:moe.algorithm:Best expert Linear_Matern32 accuracy=0.00000000000000019984014443252818
INF

INFO:ego.egor:Reject 1/1 point too close to previous ones
INFO:moe.algorithm:Best expert Linear_AbsoluteExponential accuracy=0.0000000000000006883382752675971
INFO:moe.algorithm:Best expert -> Linear_AbsoluteExponential
INFO:moe.algorithm:Best expert Linear_SquaredExponential accuracy=0.1860013107773494
INFO:moe.algorithm:Best expert -> Linear_SquaredExponential
INFO:moe.algorithm:Best expert Quadratic_SquaredExponential accuracy=0.2828762712939571
INFO:moe.algorithm:Best expert -> Quadratic_SquaredExponential
INFO:ego.egor:Reject 1/1 point too close to previous ones
INFO:moe.algorithm:Best expert Linear_AbsoluteExponential accuracy=0.0000000000000006883382752675971
INFO:moe.algorithm:Best expert -> Linear_AbsoluteExponential
INFO:moe.algorithm:Best expert Linear_SquaredExponential accuracy=0.1860013107773494
INFO:moe.algorithm:Best expert -> Linear_SquaredExponential
INFO:moe.algorithm:Best expert Quadratic_SquaredExponential accuracy=0.2828762712939571
INFO:moe.algorithm:Best expert 

Optimization f=[-3.932803705812783, -0.08929177637534513, -1.67437722939286] at [1.8313879796937838, 2.1014157261189994]


In [6]:
help(egor)

Help on Optimizer in module builtins object:

class Optimizer(object)
 |  Optimizer(fun, xlimits, n_cstr=0, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, n_parallel=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  Parameters
 |  
 |      fun: array[n, nx]) -> array[n, ny]
 |          the function to be minimized
 |          fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |             obj is the objective function [n, nx] -> [n, 1]
 |             cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |             an k the number of constraints (n_cstr)
 |             hence ny = 1 (obj) + k (cstrs)
 |          cstr functions are expected be negative (<=0) at the optimum.
 |  
 |      n_cstr (int):
 |          the number of constraint functions.
 |  
 |      xlimits (array[nx, 2]):
 |          Bounds of th nx components of the input x (eg. [[lower_1, upper_1], ..., [lower_nx, upper_nx]])
 |  
 |   